In [ ]:
import json
import pandas as pd
import geopandas as gpd
import numpy as np

In [ ]:
path = "./geo_data_trees.geojson"

In [ ]:
with open(path, 'r') as f:
    data = json.load(f)

In [ ]:
df = []
for i in range(len(data["features"])):
    props = data["features"][i]["properties"]
    df.append(props)

In [ ]:
# Normalize the data
df = pd.json_normalize(df)

In [ ]:
df.head(1)

In [ ]:
df['Crown Width (m)'].head(1)

In [ ]:
# Rename columns
df.columns = df.columns.str.strip().str.lower().str.replace('[\s\/]', '_').str.replace('[()]', '')

In [ ]:
# The last row is the total
df[-1:]

In [ ]:
# Remove the last row
df = df.drop(df.index[-1])

In [ ]:
# Check for null values
df.isnull().sum()

In [ ]:
# Drop empty columns
df.drop(['carbon_avoided_kg_yr', 'carbon_avoided_eur_yr', 'energy_savings_eur_yr'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
# display(df.dtypes)

In [ ]:
# Convert tree_id to int
df[df.columns[2:]] = df[df.columns[2:]].astype(float)
df['tree_id'] = df['tree_id'].astype(int)
df['name'] = df['name'].astype('string')
# display(df.dtypes)

In [ ]:
df.head()

In [ ]:
# df.name = df.name.str.replace('(^|\s)(\S)', lambda x: x.group(1) + x.group(2).upper())

In [ ]:
# df.head(1)

In [ ]:
# result = df.to_json(orient='split')

In [ ]:
# parsed = json.loads(result)
# print(parsed["data"][0])

## Task 1

In [ ]:
task1 = df.groupby('name')['name'].count().sort_values(ascending=False).reset_index(name='count')
task1_mean = df.groupby('name')['canopy_cover_m2'].mean().reset_index(name='mean_canopy_size')
t1 = task1.merge(task1_mean, on='name')
t1 = t1.round(2)
t1.head()

In [ ]:
t1.to_csv('task1.csv', index=False)

In [ ]:
# t1.to_json('task1.json', orient='split', index=False)

In [ ]:
# import csv
# with open('task1.csv', 'r') as file:
#     reader = csv.reader(file)
#     for row in reader:
#         print(row)

## Task 2

In [ ]:
path = './circoscrizioni.json'
with open(path, 'r') as f:
    data = json.load(f)

In [ ]:
gdf = gpd.GeoDataFrame.from_features(data["features"])
gdf.head()

In [ ]:
# task2 = pd.DataFrame(data, columns=['district', task1['name'][0], task1['name'][1], task1['name'][2], task1['name'][3],
#                                     task1['name'][4]], 'others')

In [ ]:
columns_names = np.array(['district', task1['name'][0], task1['name'][1], task1['name'][2], task1['name'][3],
                          task1['name'][4], 'others'])

In [ ]:
#  TEST
np.argmax(columns_names == task1['name'][3])

In [ ]:
# TEST
'test' in columns_names

In [ ]:
gdf.nome.count()

In [ ]:
# task2.head()

In [ ]:
gdf2 = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

In [ ]:
# gdf.geometry[0].contains(gdf2.geometry[11])

In [ ]:
points_within = gdf2.sjoin(gdf, predicate='within')

In [ ]:
points_within.head()

In [ ]:
points_within['name'].head()

In [ ]:
districts = []
for i in range(0, len(gdf.nome)):
    districts.append(gdf.nome[i])

In [ ]:
districts = np.array(districts)

In [ ]:
np.argmax('MEANO' == districts)

In [ ]:
('A' in districts)

In [ ]:
r = np.zeros((gdf.nome.count(), len(columns_names)))
for row in points_within.iterrows():
    index_row = np.argmax(districts == row[1]['nome'])
    if row[1]['name'] in columns_names:
        index = np.argmax(columns_names == row[1]['name'])
        r[index_row, index] += 1
    else:
        r[index_row, 6] += 1

In [ ]:
task2 = pd.DataFrame(data=r,
                     columns=columns_names)

In [ ]:
for i in range(len(districts)):
    task2['district'][i] = districts[i]

In [ ]:
task2.columns[1:]

In [ ]:
task2[task2.columns[1:]] = task2[task2.columns[1:]].applymap(int)
task2.head()

In [ ]:
# task2.iloc[:, 1:]

In [ ]:
task2['tot'] = task2.iloc[:, 1:].sum(axis=1)

In [ ]:
# task2.head()

In [ ]:
task2.to_csv('task2.csv', index=False)

In [ ]:
# task3 = task2.transpose(copy=True)

In [ ]:
# task3.to_csv('task3.csv', index=True)

In [ ]:
df.head()

In [ ]:
data2 = df.groupby('name')['name'].count().sort_values(ascending=False).reset_index(name='count')
# aa2_mean = df.groupby('name')['canopy_cover_m2'].mean().reset_index(name='mean_canopy_size')
# a2 = aa2.merge(task1_mean, on='name')
# a2 = a2.round(2)
data2.head(10)
print(data2['name'][:10])

In [ ]:
N = 6
top6 = df.groupby('name')['name'].count().sort_values(ascending=False).reset_index(name='count')
# task1_mean = df.groupby('name')['canopy_cover_m2'].mean().reset_index(name='mean_canopy_size')
# t1 = task1.merge(task1_mean, on='name')
# t1 = t1.round(2)
top6_names = top6.head(N)['name']

In [ ]:
top6_data = df[['name', 'height_m', 'crown_width_m', 'dbh_cm', 'gross_carbon_sequestration_kg_yr']]
top6_data = top6_data[top6_data['name'].isin(top6_names)]

In [ ]:
# top6_names = np.array(top6_names)

In [ ]:
# top6_data.sort_values(ascending=True, by='name').head()
## CATEGORICAL SORTING
top6_data['name'] = pd.Categorical(top6_data['name'], categories=top6_names)
top6_data = top6_data.sort_values(by='name')

In [ ]:
top6_data.head()

In [ ]:
top6_data.to_csv('top6_data.csv', index=False)

In [ ]:
top6_data[top6_data['name'] == 'Celtis australis'].describe()

In [ ]:
top6_data.describe()

In [ ]:
# id = 8074
# df[df['tree_id'] == id]

In [ ]:
# CO2 assorbita = gros_carbon_sequestration_kg_yr
# diametro tronco = dbh_cm
# altezza = height_m
# diametro chioma = crown_width_m
# copertura foglie = leaf_area_m2
# copertura chioma = canopy_cover_m2
# produzione ossigeno = oxygen_production_kg_yr
# inquinamento rimosso = pollution_removal_g_yr

In [ ]:
# top6_data.plot.scatter('dbh_cm', 'gross_carbon_sequestration_kg_yr', c='name', colormap='tab10')

# Assignment 3

In [ ]:
# GEOJSON - GEOPANDAS
# path_geo = "./geo_data_trees.geojson"
# gdf3 = gpd.read_file(path_geo)

In [ ]:
points_within.head()

In [ ]:
trento_trees = points_within[['tree_id', 'name', 'oxygen_production_kg_yr', 'canopy_cover_m2', 'geometry', 'nome']]

In [ ]:
trento_trees.head()

In [ ]:
trento = gdf[['geometry', 'area', 'nome']]

In [ ]:
trento.head()

In [ ]:
# trento_trees.to_file("trento_trees.geojson", driver='GeoJSON')
# trento.to_file("trento.geojson", driver='GeoJSON')

In [ ]:
district_count = trento_trees.groupby('nome')['nome'].count().sort_values(ascending=False).reset_index(name='count')

In [ ]:
# district_count.head()

In [ ]:
trento = trento.join(district_count.set_index(["nome"]), on='nome')

In [ ]:
canopy_cover = trento_trees.groupby('nome')['canopy_cover_m2'].sum().sort_values(ascending=False).reset_index(
    name='canopy_cover')

In [ ]:
oxygen = trento_trees.groupby('nome')['oxygen_production_kg_yr'].sum().sort_values(ascending=False).reset_index(
    name='oxygen')

In [ ]:
trento = trento.join(canopy_cover.set_index(["nome"]), on='nome')
trento = trento.join(oxygen.set_index(["nome"]), on='nome')

In [ ]:
# trento.head()

In [ ]:
trento['density'] = trento['canopy_cover'] / trento['area']

In [ ]:
trento.head()

In [ ]:
trento.to_file("trento.geojson", driver='GeoJSON')

In [ ]:
trento_trees[['name', 'geometry', 'nome']].to_file("trento_trees.geojson", driver='GeoJSON')

# Assignment 4

In [ ]:
path4 = "./salorno.csv"

In [ ]:
a4 = pd.read_csv(path4)

In [ ]:
# a4.head()

In [ ]:
# Check for null values
a4.isnull().sum()

In [ ]:
# Drop missing values
a4 = a4.dropna()

In [ ]:
# Convert date to datetime format
a4['date'] = pd.to_datetime(a4['date'], dayfirst=True)

In [ ]:
# a4.head()

In [ ]:
a4['year'] = a4['date'].map(lambda x: x.year)
a4['month'] = a4['date'].map(lambda x: x.month)

In [ ]:
# a4.head()

In [ ]:
a4 = a4.loc[a4['year'].isin([1993, 1997, 2001, 2005, 2009, 2013, 2017, 2021])]

In [ ]:
# a4.head()

In [ ]:
# Reset index
a4.reset_index(drop=True, inplace=True)

In [ ]:
# Task 3
a4_task3 = a4.copy()
a4_task3['day'] = a4_task3['date'].map(lambda x: x.day)
a4_task3.drop('date', axis=1, inplace=True)
a4_task3[a4_task3.columns[:2]] = a4_task3[a4_task3.columns[:2]].astype(float)

In [ ]:
a4_task3.to_csv('assignment4_task3.csv', index=False)

In [ ]:
a4.drop('date', axis=1, inplace=True)

In [ ]:
# a4.dtypes

In [ ]:
# a4.columns[:2]

In [ ]:
a4[a4.columns[:2]] = a4[a4.columns[:2]].astype(float)

In [ ]:
a4.head()

In [ ]:
# Calculate the daily mean
a4 = a4.assign(daily_mean=a4[['min','max']].mean(axis=1))

In [ ]:
a4.head()

In [ ]:
# Aggregate min, max, mean
a4 = a4.groupby(['year', 'month']).agg(min=('min', 'min'), max=('max', 'max'), mean=('daily_mean', 'mean')).reset_index()

In [ ]:
# Round the results
a4['mean'] = a4['mean'].round(2)

In [ ]:
# Check
a4.loc[a4['year'] == 1993]

In [ ]:
a4.to_csv('assignment4.csv', index=False)

# Assignment 5

In [ ]:
top10 = df.groupby('name')['name'].count().sort_values(ascending=False).reset_index(name='count')
top10_names = top10.head(10)['name']

In [ ]:
top10_names.tolist()

In [ ]:
a5 = df[['name', 'carbon_storage_kg', 'total_annual_benefits_eur_yr']].copy() # carbon_storage_eur ??

In [ ]:
a5['name'] = df['name'].apply(lambda x: x if x in top10_names.tolist() else 'Others')

In [ ]:
a5 = a5.groupby('name')[['carbon_storage_kg', 'total_annual_benefits_eur_yr']].sum()

In [ ]:
# a5.to_csv('assignment5.csv')
a5.round(0).astype(int).to_csv('assignment5.csv')

In [ ]:
# df[df['tree_id'] == 12380]